In [ ]:
# todo: check the model cfg: augmentations, policy

In [1]:
import torch
from torch import nn
from utils import parse_cfg
from darknet import Darknet

In [2]:
darknet = Darknet('cfg/yolov3.cfg')

to del, this message should be printed 3ish times
to del, this message should be printed 3ish times
to del, this message should be printed 3ish times
make_layers returns net_info as well. check whether it"s necessary


In [9]:
self_layer_info[1]

{'activation': 'leaky',
 'batch_normalize': '1',
 'filters': '64',
 'name': 'convolutional',
 'pad': '1',
 'size': '3',
 'stride': '2'}

In [25]:
t = []
x = torch.randn((2, 3, 608, 608))
t.append(x)
print(t)
x = x+ 1
print(t)

[tensor([[[[-0.9665,  0.5351, -0.3072,  ...,  1.1621, -0.1265,  0.6186],
          [ 0.6774,  0.2178, -0.0142,  ...,  0.3335,  0.8363,  2.2494],
          [ 0.6761, -0.6914, -2.3339,  ...,  0.7657, -1.5950, -0.4754],
          ...,
          [-1.5157,  0.0293, -0.2095,  ...,  1.9457, -0.9569,  0.0657],
          [-0.9379,  1.0446, -1.2633,  ..., -0.3359,  0.9698,  0.0881],
          [ 0.5073, -0.1258,  0.0168,  ...,  0.1998,  0.2698, -0.4370]],

         [[ 0.0967,  1.8158, -0.1086,  ...,  1.5871,  1.2152,  0.9816],
          [ 0.0688, -0.7439,  0.3140,  ..., -0.7040, -1.7047,  1.5185],
          [ 1.2309, -0.6477,  2.7795,  ...,  1.1287,  1.0372, -0.8416],
          ...,
          [-2.2436, -1.0690,  0.4535,  ..., -0.5805,  0.0470,  0.3810],
          [-0.8969, -1.4079, -0.5210,  ..., -0.9237,  0.3257,  1.2608],
          [ 0.4565,  1.2727,  0.8348,  ...,  1.6407,  0.1382, -2.1700]],

         [[ 0.7127, -1.0983,  2.0368,  ...,  1.2454,  0.3194, -0.7733],
          [ 0.3774,  0.0611, 

In [26]:
# todo forward use both info and modules 
x = torch.randn((2, 3, 608, 608))

self_layer_list = darknet.layers_list
self_layer_info = darknet.layers_info[1:]

# cache the outputs for route and shortcut layers
outputs = []

for i, layer in enumerate(self_layer_list):
    
    name = self_layer_info[i]
    # TODO: append something after each layer!!!!
    if name in ['convolutional', 'upsample']:
        x = layer(x)
        outputs.append(x)
        
    elif name == 'shortcut':
        # index which is used for shortcut (usually '-3')
        x = outputs[-1] + outputs[layer.frm]
#         x = x + outputs[layer.frm]
        outputs.append(None)
        
    elif name == 'route':
        to_cat = [outputs[route_idx] for route_idx in layer.routes]
        x = torch.cat(to_cat, dim=1)
        
    elif name == 'yolo':
        # TODO
        pass
        

SyntaxError: invalid syntax (<ipython-input-26-9ad423323e74>, line 23)

In [7]:
self_layer_list[0]

Sequential(
  (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky_0): LeakyReLU(negative_slope=0.1)
)